In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = (512, 512)
IMAGE_DIR = Path("/home/anawat/JJ/Chula/Chess2PGN/data")

# Chess board part configs
BILATERAL_FILTER_DIAMETER = 3
BILATERAL_FILTER_SIGMA_COLOR = 75
BILATERAL_FILTER_SIGMA_SPACE = 75